# Simple Alfalfa Example

## Setup
1. [Alfalfa stack deployed](https://github.com/NREL/alfalfa/wiki/Deployment) locally or have the url for the alfalfa host you are using


In [ ]:
import datetime
from alfalfa_client.alfalfa_client import AlfalfaClient
from pprint import pprint

### Create new alfalfa client object
If you are not hosting the alfalfa server yourself replace the `host` with the one of your server without a trailing slash (this is a known bug and will eventially be fixed)

In [ ]:
ac = AlfalfaClient(host='http://localhost')

### Define paths to models to be uploaded
Model path can be a directory (for OSWs) or a file (.zip for OSWs or .fmu)

In [ ]:
model_path = './twobldgs'

### Upload sites to alfalfa
The `ac.submit` function returns the site_id which is used to interact with that specific site over the API. A simulation is a site.


Sites can be viewed at http://localhost/sites

In [ ]:
site_id = ac.submit(model_path)
print(site_id)


### Define parameters to run the simulations

In [ ]:
# If you are using historian, you will need to search for this time period in Grafana dashboard to view results.
start_dt = datetime.datetime(2021, 7, 1, 12, 2, 0)
end_dt = datetime.datetime(2021, 7, 3, 0, 0, 0)

# For external_clock == true, API calls are used to advance the model.
# If external_clock == false, Alfalfa will handle advancing the model according to a specified timescale (timescale 1 => realtime)
params = {
    "external_clock": True,
    "start_datetime": start_dt,
    "end_datetime": end_dt
}

## Start simulations 
Note: one sim runs / worker, so if you have not scaled your local deployment to workers >= 2, the second worker won't have a chance to start and this code block will not complete.

In [ ]:
print(f"Starting site: {site_id}")
ac.start(site_id, **params)

### Get the model's input points
Get a list of all of the model's input points and their values


In [ ]:
print(f"{site_id} inputs:")
pprint(ac.get_inputs(site_id))

## Set model input point

To set an input value use the `ac.set_inputs(site_id, inputs)` function. 
- `site_id` - the id of the site returned by the `ac.submit` function
- `inputs` - a dictionary of input names and the desired values


In [ ]:
input_dict = {'Core_ZN_ZN_PSZ_AC_1_Outside_Air_Damper_CMD': 0.7}
ac.set_inputs(site_id, input_dict)

### Advance the model
12/10/2021: timestep is hardcoded to 1 minute w/in Alfalfa worker.

In [ ]:
timesteps = 5
for _ in range(timesteps):
    ac.advance(site_id)
    print(f"Model advanced to time: {ac.get_sim_time(site_id)}")

### Get model's outputs
Query the outputs of the models as well as their values

In [ ]:
print(f"{site_id} outputs:")
pprint(ac.get_outputs(site_id))

### Stop the simulation

In [ ]:
ac.stop(site_id)